In [1]:
# Dependencies
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
# Import IMDB movie data (Title Basic, Name Basic, Title Crew, Title Ratings)
# https://www.kaggle.com/tinoswe/split-python-dataframe
title_basics = pd.read_csv('../data/title.basics.tsv',delimiter='\t',encoding='utf-8-sig',low_memory=False)

# Import title ratings
title_ratings = pd.read_csv('../data/title.ratings.tsv',delimiter='\t',encoding='utf-8-sig')

# Import title crew
title_crew = pd.read_csv('../data/title.crew.tsv',delimiter='\t',encoding='utf-8-sig')

# Import name basics
name_basics = pd.read_csv('../data/name.basics.tsv',delimiter='\t',encoding='utf-8-sig')

# Import box office data from BoxOfficeMojo
box_office = pd.read_csv('../data/boxoffice.csv')

# Import Oscar data from Wikipedia: https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films
oscars = pd.read_csv('../data/oscars_cleaned.csv')

In [105]:
## REFACTOR

# Filter dataframe to exclude tv data
title_basics_filtered = title_basics[title_basics['titleType']!="tvEpisode"]

# Split out genres to new dataframe, concatonate with main movie dataframe
genres_split = title_basics_filtered["genres"].str.split(",", n=2, expand=True)
joined = pd.concat([title_basics_filtered, genres_split], axis=1, sort=False)

# Rename and drop some columns
joined = joined.rename(columns = {'tconst':'IMDB ID', 'titleType': 'Type', 'primaryTitle': 'Title', 'originalTitle': 'Title (original)', 'isAdult': 'Adult (yes/no)', 'startYear': 'Year', 'runtimeMinutes': 'Runtime (min)', 0:'Genre (Main)',1:'Genre (Sub 1)', 2:'Genre (Sub 2)'})
joined = joined.drop(columns=['endYear', 'genres'])

# Filter short films
features = joined[joined['Type']!='short']

# Filter adult movies
features = features[features['Adult (yes/no)']==0]

# Filter movies with no runtime
realmovies = features.dropna(axis='rows')

In [212]:
### LOSING DATA HERE, CHECK JOIN/MERGE

# Merge basic dataframe and rating dataframe
title_ratings_merged = realmovies.set_index('IMDB ID').join(title_ratings.set_index('tconst'))
title_ratings_merged = title_ratings_merged.rename(columns = {'averageRating': 'Rating (avg.)', 'numVotes': 'Number of votes'})

In [213]:
title_ratings_merged.count()

Type                138526
Title               138526
Title (original)    138526
Adult (yes/no)      138526
Year                138526
Runtime (min)       138526
Genre (Main)        138526
Genre (Sub 1)       138526
Genre (Sub 2)       138526
Rating (avg.)        89475
Number of votes      89475
dtype: int64

In [183]:
# Merge, drop columns we don't need
merged = box_office.join(oscars.set_index('Film'), on='title')
merged = title_ratings_merged.join(merged.set_index('title'), on='Title', rsuffix='_right')
merged = merged.drop(['Adult (yes/no)', 'Year_right', 'year', 'rank'], axis=1)

In [ ]:
# Drop tvMovie, tvSeries etc.
# Check for dupes

In [180]:
# Budget

#merged = merged.dropna(axis='rows')
#merged.head()

## STUFF NOT USING AT THE MOMENT

In [ ]:
# Calculate avg gross ($) per movie
box_plot['Avg. lifetime gross'] = box_plot['lifetime_gross']/box_plot['title']

# Reset index to get year in column
box_plot.reset_index()